# import packages and load data

In [ ]:
import pickle
from flair.data import Corpus
from flair.datasets import SentenceDataset
from sklearn.model_selection import train_test_split
from flair.embeddings import WordEmbeddings, CharacterEmbeddings,StackedEmbeddings

In [ ]:
path = '/Users/Wu/Google Drive/'
with open(path+'data/data_news_LogitsLabel_12.7k.pickle', 'rb') as handle:
    data_news = pickle.load(handle)

with open(path+'data/data_conll_LogitsLabel_12.7k.pickle', 'rb') as handle:
    data_conll = pickle.load(handle)

data = data_news+data_conll

data_train, data_test = train_test_split(data, test_size=0.2, random_state=42)
data_dev,data_test = train_test_split(data_test, test_size=0.5, random_state=42)

corpus: Corpus = Corpus(SentenceDataset(data_train),SentenceDataset(data_test),SentenceDataset(data_dev))
tag_type = 'ner'

tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)
corpus: Corpus = corpus.downsample(0.1)

In [ ]:
from KD_sequence_tagger_model import SequenceTagger
# from flair.models import SequenceTagger

embedding_types = [
    WordEmbeddings('de'),
    CharacterEmbeddings(),
]
embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=False,
                                        reproject_embeddings = 100,
                                        # use_soft_labels=True,
                                        # use_logits = True,
                                        )

#%% train the model 
from flair.trainers import ModelTrainer
trainer: ModelTrainer = ModelTrainer(tagger, corpus)

trainer.train('resources/taggers/test2.5k_flair_glove',
              learning_rate=0.1,
              mini_batch_size=10,
              max_epochs=15,
              checkpoint=True,
              )

#%% continue training the model
# checkpoint = 'resources/taggers/test/checkpoint.pt'
# trainer = ModelTrainer.load_checkpoint(checkpoint, corpus) 
# trainer.train(path+'resources/taggers/test60ep',
#               learning_rate=0.05,
#               mini_batch_size=10,
#               max_epochs=50,
#               checkpoint=True) 